In [23]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.actions.action_builder import ActionBuilder
from selenium.webdriver.common.actions.key_input import KeyInput
from selenium.webdriver.common.actions.pointer_input import PointerInput


# System requiremenmts 
import time
import random
import datetime as dt

# Data Management 
import pandas as pd

# Web scrapping functions 
import src.web_minning as wm

# Geodata 
import geopandas as gpd 
import folium 

In [3]:
RAW_DATA = "./data/raw/"

In [7]:
df_mercados = gpd.read_file(RAW_DATA + "ubi_tianguis.json")
df_places = gpd.read_file(RAW_DATA + "places_silver_ns.geojson")

In [ ]:
df_places.query(""" start_dt != '' and end_dt """)

,href,long_description,add_label_used,ind_found,lat,lon,osm_name_label,start_dt,end_dt,date_label,time_label,dt_confidence,time_finder_labels,geometry
8,https://www.tiktok.com/@comelones_mex/video/75...,Las mejores roscas de reyes en la CDMX 2026\nD...,CDMX 2026,True,19.432630,-99.133178,"Ciudad de México, Cuauhtémoc, Ciudad de México...",2026-01-09 05:00:00-06:00,None,None,5,0.5740000000000001,"('date_imputed', 'keyword')",POINT (-99.13318 19.43263)
11,https://www.tiktok.com/@rutadeviajemx/video/74...,Descubre el Museo Nacional de Arte MUNAL en CD...,"Siqueiros, Cdmx",True,19.308108,-99.169698,"Siqueiros, 7, Camino Rojo, Rinconada de los Pi...",2026-01-09 10:00:00-06:00,None,None,10 am,0.7075,"('date_imputed', 'icon', 'keyword')",POINT (-99.1697 19.30811)
18,https://www.tiktok.com/@gumadetona/video/75244...,Atracción CDMX: Juegos y Diversión en Parques\...,"Parques, Cdmx",True,19.299984,-99.190255,"Colonia Parques del Pedregal, Ciudad de México...",2026-01-09 05:00:00-06:00,None,None,5,0.5740000000000001,"('date_imputed', 'keyword')",POINT (-99.19025 19.29998)
30,https://www.tiktok.com/@gumadetona/video/75244...,Atracción CDMX: Juegos y Diversión en Parques\...,"Parques, Cdmx",True,19.299984,-99.190255,"Colonia Parques del Pedregal, Ciudad de México...",2026-01-09 05:00:00-06:00,None,None,5,0.5740000000000001,"('date_imputed', 'keyword')",POINT (-99.19025 19.29998)
38,https://www.tiktok.com/@marieldelcoy/video/751...,Cosas que hacer en la CDMX: 15 actividades imp...,"la ciudad, Cdmx",True,19.434235,-99.073312,Aeropuerto Internacional de la Ciudad de Méxic...,2026-01-09 15:00:00-06:00,None,None,15,0.5740000000000001,"('date_imputed', 'keyword')",POINT (-99.07331 19.43423)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
346,https://www.tiktok.com/@jclavozdelbarrio/video...,Festival de luces en el Zócalo de la Ciudad de...,"el Zócalo, Cdmx",True,19.433864,-99.134245,"El Balcón del Zócalo, 61, Calle 5 de Mayo, Cen...",2026-01-03 03:00:00-06:00,None,3 de enero,3,0.9430000000000001,"('event_hint', 'keyword')",POINT (-99.13424 19.43386)
347,https://www.tiktok.com/@tecolotediario/video/7...,Festival de la Rosca de Reyes 2026 en CDMX\nDi...,"Reyes, Cdmx",True,19.246267,-99.054389,"Colonia Los Reyes, La Espejera, Xochimilco, Ci...",2026-01-04 03:00:00-06:00,None,4 de enero,3,0.9430000000000001,"('event_hint', 'keyword')",POINT (-99.05439 19.24627)
349,https://www.tiktok.com/@brjula.informativ/vide...,Concierto Gratis de Víctimas del Doctor Cerebr...,"el Zócalo, Cdmx",True,19.433864,-99.134245,"El Balcón del Zócalo, 61, Calle 5 de Mayo, Cen...",2026-12-30 00:00:00-06:00,None,30 de diciembre,None,0.718,"('event_hint', 'keyword', 'time_missing')",POINT (-99.13424 19.43386)
350,https://www.tiktok.com/@fidelomega498/video/75...,Los Amantes de Lola: Concierto en el Zócalo CD...,"el Zócalo, Cdmx",True,19.433864,-99.134245,"El Balcón del Zócalo, 61, Calle 5 de Mayo, Cen...",2026-01-02 02:00:00-06:00,None,2 de enero,2,0.9430000000000001,"('event_hint', 'keyword')",POINT (-99.13424 19.43386)


In [5]:
df_mercados.loc[2,"geometry"].coords[:][0][0]

-99.19793561503671

In [ ]:


m = folium.Map(tiles="cartodb positron")

df = df_places.copy()

for index, row in df.iterrows():
    
    coords = [row['geometry'].coords[:][0][1], row['geometry'].coords[:][0][0]]
    
    folium.CircleMarker(
        location=coords, 
        radius = 5, 
        popup = f"""
    <div style="
        width:260px;
        font-family: Arial, sans-serif;
        border:1px solid #ddd;
        border-radius:10px;
        box-shadow:0 2px 6px rgba(0,0,0,0.15);
        overflow:hidden;
    ">

    <!-- Card header -->
    <div style="
        background:#f5f5f5;
        padding:8px 10px;
        font-weight:600;
        font-size:13px;
        text-align:center;
    ">
        <a href="{row['href']}" target="_blank"
        style="text-decoration:none;color:#111;">
        Open video
        </a>
    </div>

    <!-- Card body -->
    <div style="padding:10px;font-size:12px;color:#333;">
        <div style="margin-bottom:6px;">
        {row['long_description']} 
        </div>

        <div style="
            font-size:11px;
            color:#666;
            border-top:1px solid #eee;
            padding-top:6px;
        ">
        {row['add_label_used']} 
        </div>
    </div>

    </div>
    """


    ).add_to(m)

m

In [31]:


# Save
m.save(RAW_DATA + "mapa_test.html")





In [29]:
desc_text

'Carnitas El Cherán · Ciudad de México\nchuladademaizprieto_\n· Hace 6 d\nDE LAS MEJORES CARNITAS QUE YO HE PROBADO! Sin lugar a dudas un super lugar en CDMX! @el_cheran Que Chulada!\n\n📍Av Arcos de Belén 31-L3, Colonia Centro, Centro, Cuauhtémoc, 06000 Ciudad de México, CDMX\nMejores Carnitas en CDMX: El Cheran\nDescubre las mejores carnitas en CDMX en El Cheran y disfruta de una experiencia culinaria única. ¡No te lo pierdas! #elcheran #Carnitas #ComidaCDMX\nKeywords: mejores carnitas en CDMX, dónde comer en CDMX, restaurantes en CDMX, comida típica en CDMX, El Cheran carnitas, lugares para comer en CDMX, carnitas recetas, comida callejera CDMX, gastronomía mexicana, comida en Cuauhtémoc\n\nmenos\nCumbia Caliente - Fito Olivares Y Su Grupo'